# Testing PINN implementations

- Autograd vs. PyTorch
- Gradient descent methods (optimization methods)
- Learning rates (epochs)

## Setup and imports


In [ ]:
# Notebook setup: add repo root to path and import project helpers.
import sys, os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

from src import *

## Configuration


In [ ]:
##################################################################################
# CONFIGURATION INFORMATION : 
##################################################################################
# NETWORK ARCITECTURE : 
num_hidden_neurons = [2,2]  # A simple network just for testing
opt_met            = ['GD', 'GD-adam', 'SGD-adam']
# HYPERPARAMETERS : 
epochs         = 100  
learning_rate  = [0.1, 0.01, 0.001, 0.0001]
seed           = 36  # Most similar seed for numpy.random.seed and PyTorch.manual_seed
# ONLY FOR SGD : 
batch_fraction = 0.1
# GRID RESOLUTION : 
Nx = 10    # Smaller grid resolution for testing
Nt = 10    # Smaller grid resolution for testing

batch_size = 10
# MAKE X AND T : 
t, x, t_torch, x_torch, _ = create_t_and_x_batch_size(Nx,
                                                      Nt,
                                                      batch_fraction)
# CONFIGURATION INFORMATION : 
config = make_config(Nx,
                     Nt, 
                     num_hidden_neurons,
                     opt_met, 
                     epochs, 
                     learning_rate, 
                     batch_fraction,
                     batch_size,
                     seed,
                     verbose=True)

## Training helpers


In [ ]:
##################################################################################
# FUNCTIONS FOR TESTING GRADIENT DECENT METHODS AND LEARNING RATES : 
##################################################################################
def compare_cost_and_time_A(optimization_method,learning_rate):
    start_time = time.time()
    P, history = train_PINN_autograd(x,
                                     t,
                                     num_hidden_neurons,
                                     epochs,
                                     learning_rate,
                                     'tanh',
                                     seed,
                                     optimization_method=optimization_method, # 'GD', 'GD-adam', 'SGD-adam'
                                     batch_size=batch_size,
                                     shuffle=True,
                                     replacement=False,
                                     verbose=False,
                                     debug=False)
    end_time = time.time()
    tot_time = end_time - start_time
    return tot_time, history, P
def compare_cost_and_time_P(optimization_method,learning_rate):
    start_time = time.time()
    P, history = train_PINN_pyTorch(x_torch,
                                    t_torch,
                                    num_hidden_neurons,
                                    epochs,
                                    learning_rate,
                                    'tanh',
                                    seed,
                                    optimization_method=optimization_method, # 'GD', 'GD-adam', 'SGD-adam'
                                    batch_size=batch_size,
                                    shuffle=True,
                                    replacement=False,
                                    verbose=False,
                                    debug=False)
    end_time = time.time()
    tot_time = end_time - start_time
    return tot_time, history, P

## Run training sweeps


In [ ]:
##################################################################################
# TESTING PINN-IMPLEMENTATINOS : 
##################################################################################
learning_rates = learning_rate
# RUN TEST : 
results_A = {}
results_P = {}
for lr in learning_rates:
    print('----------------------------------')
    print(f'Learning rate: {lr}')
    print('----------------------------------')
    res_A = {}
    res_P = {}
    for opt in opt_met:
        print(f'Optimization method: {opt}')
        # Run the comparison for autograd and pytorch
        print('Autograd')
        time_A, hist_A, PA = compare_cost_and_time_A(opt, lr)
        print('PyTorch')
        time_P, hist_P, PP = compare_cost_and_time_P(opt, lr)
        # Save the results in dict, to plot later
        res_A[opt] = {'time': time_A, 'history': hist_A, 'P': PA}
        res_P[opt] = {'time': time_P, 'history': hist_P, 'P': PP}
        print()
    results_A[lr] = res_A
    results_P[lr] = res_P
    print()

## Plot: final cost vs learning rate


In [ ]:
# Final cost comparison plot between autograd and pytorch, for optimization methods and learning rates
# Learning rates on x-axis, final cost on y-axis
# NB! ONLY PLOTTING THE EXECUTION TIME FOR THE FIRST LEARNING RATE
plt.figure(figsize=(10,6))
for opt in opt_met:
    final_costs_A = [results_A[lr][opt]['history']['cost'][-1] for lr in learning_rates]
    final_costs_P = [results_P[lr][opt]['history']['cost'][-1] for lr in learning_rates]
    time_A = [results_A[lr][opt]['time'] for lr in learning_rates]
    time_P = [results_P[lr][opt]['time'] for lr in learning_rates]
    plt.plot(learning_rates, final_costs_A, marker='o', label=f'{opt} Time: {time_A[0]:.2f}s', linestyle='--')
    plt.plot(learning_rates, final_costs_P, marker='o', label=f'{opt} Time: {time_P[0]:.2f}s', linestyle='-')
plt.xlabel('Learning Rate')
plt.ylabel('Final Cost')
# log scale for x-axis
plt.xscale('log')
#plt.yscale('log')
plt.legend()
plt.title(f'Final Cost Comparison of learning rates, with epoch={epochs}')
save_fig('test0_final_cost_vs_lr')
plt.show()

## Plot: cost history comparisons


In [ ]:
# Plot all the results, comparing autograd and pytorch, for different optimization methods
plt.figure(figsize=(10,6))
for learning_rate in learning_rates:
    plt.figure(figsize=(10,6))
    lr_slug = str(learning_rate).replace('.', 'p')
    for opt in opt_met: 
        plt.plot(results_A[learning_rate][opt]['history']['epoch'], results_A[learning_rate][opt]['history']['cost'], label=f'{opt} Time: {results_A[learning_rate][opt]["time"]:.2f}s', linestyle='--')
        plt.plot(results_P[learning_rate][opt]['history']['epoch'], results_P[learning_rate][opt]['history']['cost'], label=f'{opt} Time: {results_P[learning_rate][opt]["time"]:.2f}s', linestyle='-')
    #plt.yscale('log')
    plt.xlabel('Epochs')
    plt.ylabel('Cost')
    plt.legend()
    plt.title('Comparison of Autograd and PyTorch PINN Training {learning_rate='+str(learning_rate)+'}')
    save_fig(f'test0_cost_history_lr_{lr_slug}')
    plt.show()